In [15]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import imageio
from tqdm import tqdm

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler

from torchvision import models, transforms
from torchvision import transforms, utils
from torchvision import datasets

from torchsummary import summary


from mm_patch.data import PatchesDataset
import mm_patch.transforms
from mm_patch.utils import image_from_index



In [16]:
# Transformations to be compatible with Densenet-121 from NYU paper.
# Note I am using mean and std as recommended in Pytorch. Maybe calculating the dataset statistics is better.
composed = transforms.Compose([ 
#                                 mm_patch.transforms.ToImage(),
                                transforms.ToTensor(),
                                mm_patch.transforms.Scale(),
#                                 mm_patch.transforms.GrayToRGB(),
                                transforms.Normalize(mean=[0.485], std=[0.229])
                            ])

In [17]:
def read_image_png(file_name):
    image = np.array(imageio.imread(file_name)).astype(np.int32)
    return image

patches = datasets.ImageFolder('../patches_images/patches_CRO_23072019/', transform=composed, target_transform=None, loader=read_image_png)

In [18]:
# # Set dataset
# patches = PatchesDataset(data_path='./patches_256x256.pkl', transform = composed)

# Dataloader parameters
batch_size = 8
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(patches)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

# this train loader is to have random access
loader = torch.utils.data.DataLoader(patches, batch_size=batch_size, 
                                            num_workers=10)
train_loader = torch.utils.data.DataLoader(patches, batch_size=batch_size, 
                                           sampler=train_sampler, num_workers=10)
validation_loader = torch.utils.data.DataLoader(patches, batch_size=batch_size,
                                                sampler=valid_sampler, num_workers=10)

In [19]:
images, _ = next(iter(loader))
print(images[0].shape)

torch.Size([1, 256, 256])


### Model

In [20]:
import torch.nn as nn
import torch.nn.functional as F

# define the NN architecture
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        
        ## encoder layers ##
        self.conv1 = nn.Conv2d(1, 16, 3, padding = 1)
        self.conv2 = nn.Conv2d(16, 4, 3, padding = 1)
        self.conv1a = nn.Conv2d(16, 16, 3, padding = 1)
        self.pool = nn.MaxPool2d(2, 2)
        
        ## decoder layers ##
        self.t_conv1 = nn.ConvTranspose2d(4, 16, 2, stride=2)
        self.t_conv2 = nn.ConvTranspose2d(16, 1, 2, stride=2)
        
        ## a kernel of 2 and a stride of 2 will increase the spatial dims by 2
        self.t_conv1 = nn.ConvTranspose2d(4, 16, 2, stride=2)

        
    def forward(self, x):
        ## encode ##
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv1a(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)

        ## decode ##
        x = F.relu(self.t_conv1(x))
        x = torch.sigmoid(self.t_conv2(x))
                
        return x

# initialize the NN
model = ConvAutoencoder()
# print(model)

In [21]:
torch.cuda.is_available()

True

In [22]:
if torch.has_cudnn:
    device = torch.device("cuda:{}".format('0'))
else:
    device = torch.device("cpu")

# device = torch.device('cpu')
model = model.to(device)
# F: sets model in evaluation mode. It has an effect in certain modules: e.g. Dropout or BatchNorm Layers
# model.eval()

In [24]:
# from torchsummary import summary
# summary(your_model, input_size=(channels, H, W))
summary(model, input_size=(1, 256, 256), device = 'cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 256, 256]             160
            Conv2d-2         [-1, 16, 256, 256]           2,320
         MaxPool2d-3         [-1, 16, 128, 128]               0
            Conv2d-4          [-1, 4, 128, 128]             580
         MaxPool2d-5            [-1, 4, 64, 64]               0
   ConvTranspose2d-6         [-1, 16, 128, 128]             272
   ConvTranspose2d-7          [-1, 1, 256, 256]              65
Total params: 3,397
Trainable params: 3,397
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.25
Forward/backward pass size (MB): 21.12
Params size (MB): 0.01
Estimated Total Size (MB): 21.39
----------------------------------------------------------------


In [25]:
# # Example of using Sequential with OrderedDict
# model = nn.Sequential(OrderedDict([
#           ('conv1', nn.Conv2d(1,20,5)),
#           ('relu1', nn.ReLU()),
#           ('conv2', nn.Conv2d(20,64,5)),
#           ('relu2', nn.ReLU())
#         ]))


In [26]:
# loss function
criterion = nn.MSELoss()

# loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [1]:
# number of epochs to train the model
n_epochs = 30

for epoch in range(1, n_epochs+1):
    # monitor training loss
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    for data in train_loader:
        # _ stands in for labels, here
        # no need to flatten images
        images, _ = data[0].to(device), data[1]
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        outputs = model(images)
        # calculate the loss
        loss = criterion(outputs, images)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*images.size(0)
            
    # print avg training statistics 
    train_loss = train_loss/len(train_loader)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch, 
        train_loss
        ))

NameError: name 'train_loader' is not defined

In [ ]:
# obtain one batch of test images
dataiter = iter(test_loader)
images, labels = dataiter.next()

# get sample outputs
output = model(images.to(device=device))
# prep images for display
images = images.cpu().numpy()

# output is resized into a batch of iages
output = output.view(batch_size, 1, 28, 28)
# use detach when it's an output that requires_grad
output = output.detach().cpu().numpy()

# plot the first ten input images and then reconstructed images
fig, axes = plt.subplots(nrows=2, ncols=10, sharex=True, sharey=True, figsize=(25,4))

# input images on top row, reconstructions on bottom
for images, row in zip([images, output], axes):
    for img, ax in zip(images, row):
        ax.imshow(np.squeeze(img), cmap='gray')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)